# Toy Problem for RL Pathfinding

This notebook is a simplified exploration of an RL-based pathfinding algorithm. In a 10x10 repeating grid, we train an agent to go from the start point (0, 0) to the end point (9, 9) using the $Q$-learning algorithm.

### Fundamentals of RL
Reinforcement-Learning is a technique in machine learning where an agent is tasked with interacting within a specified environment. The agent takes an action $a$ during the current state of the environment $s$. The optimal action $a$ is decided by maximising a reward.

### Learning Algorithms
The learning algorithm that we'll be using is Q-learning defined as:

$$
Q_\textrm{new}(s, a) \leftarrow (1-\alpha)Q(s,a)+\alpha(r+\gamma\cdot \max_{a^\prime}Q(s^\prime, a^\prime))
$$

- $Q(s, a)$ is the current $Q$ value, aka the quality.
- $\alpha$ is the learning rate
- $r$ is the reward contribution from taking action $a$
- $\gamma$ is the discount factor for how much we value future rewards
- $\max_{a^\prime}Q(s^\prime, a^\prime)$ returns the maximum $Q$-value in the next state $s^\prime$ having taken action $a^\prime$.

From an applied mathematics background the Q-learning algorithm reminds me of classical operational research optimisation schemes.

In [3]:
import numpy as np

## The World

In [4]:
grid_size = 10
grid = np.zeros(shape=(grid_size, grid_size))

start = (0, 0)
end = (9, 9)

human_map = {'Up': 0, 'Down': 1, 'Left': 2, 'Right': 3}
agent_map = {
    0 : (-1, 0),
    1 : (1, 0),
    2 : (0,-1),
    3 : (0,1)}

## The Agent

In [5]:
alpha = 0.1
gamma = 0.9

In [6]:
class Agent:
    def __init__(self, curr_pos:tuple =start):
        self.curr_pos = curr_pos
        self.Q_table = np.zeros(shape=(grid_size, grid_size, len(human_map)))

    def move(self, action:int):
        row_change, col_change = agent_map[action]

        row_pos = self.curr_pos[0] + row_change
        row_pos %= grid_size

        col_pos = self.curr_pos[1] + col_change
        col_pos %= grid_size

        new_pos = (row_pos, col_pos)
        
        return new_pos
    
    def choose_action(self, epsilon):
        crit = np.random.uniform(0, 1)

        if crit < epsilon:
            action = np.random.choice([0, 1, 2, 3])
            return int(action)
        else:
            q_vals = self.Q_table[self.curr_pos[0], self.curr_pos[1]]
            action = np.argmax(q_vals)
            return int(action)
        
    def update_q_table(self, action, reward, new_pos):
        old_q = self.Q_table[self.curr_pos[0], self.curr_pos[1], action]
        max_q = np.max(self.Q_table[new_pos[0], new_pos[1]])
        new_q = old_q + alpha * (reward + gamma*(max_q) - old_q)
        self.Q_table[self.curr_pos[0], self.curr_pos[1], action] = new_q


### The $Q$-table

The $Q$-table is basically the registry for all our moves. The value at (3, 4, 1) is the $Q$-value of action 1, moving down, from position (3, 4). Similarly (3, 4, 3) is the $Q$-value of action 3, moving right, from position (3, 4).

This current example is very low dimensional, `(10, 10, 4)`. As such we won't need to implement a neural network to approximate the functions and can explicitly calculate them.

## Training

In [9]:
episodes = 1000

reward_end  = 100
reward_step = -1

epsilon = 1.0
epsilon_decay = 0.995

bob = Agent()

for ep in range(episodes):
    print(f"Starting Episode {ep}")
    convergence_criteria = 10000
    iter_count = 0
    bob.curr_pos = start

    while bob.curr_pos != end and iter_count <= convergence_criteria:
        iter_count += 1

        action = bob.choose_action(epsilon)
        new_pos = bob.move(action)

        if new_pos == end:
            reward = reward_end
            print(f"Agent took {iter_count} moves")
        else:
            reward = reward_step
            
        bob.update_q_table(action, reward, new_pos)
        bob.curr_pos = new_pos
        
    
    epsilon *= epsilon_decay

Starting Episode 0
Agent took 6 moves
Starting Episode 1
Agent took 138 moves
Starting Episode 2
Agent took 86 moves
Starting Episode 3
Agent took 614 moves
Starting Episode 4
Agent took 2 moves
Starting Episode 5
Agent took 114 moves
Starting Episode 6
Agent took 4 moves
Starting Episode 7
Agent took 10 moves
Starting Episode 8
Agent took 912 moves
Starting Episode 9
Agent took 2 moves
Starting Episode 10
Agent took 2 moves
Starting Episode 11
Agent took 56 moves
Starting Episode 12
Agent took 114 moves
Starting Episode 13
Agent took 2 moves
Starting Episode 14
Agent took 4 moves
Starting Episode 15
Agent took 34 moves
Starting Episode 16
Agent took 12 moves
Starting Episode 17
Agent took 74 moves
Starting Episode 18
Agent took 274 moves
Starting Episode 19
Agent took 126 moves
Starting Episode 20
Agent took 2 moves
Starting Episode 21
Agent took 8 moves
Starting Episode 22
Agent took 10 moves
Starting Episode 23
Agent took 2 moves
Starting Episode 24
Agent took 72 moves
Starting Epis

### Testing Learned Policy

In [10]:
bob.curr_pos = start
path = [start]

conv_crit = 100
iter_count = 0
while bob.curr_pos != end and iter_count < conv_crit:
    iter_count += 1
    action  = bob.choose_action(0.0)
    new_pos = bob.move(action)
    path.append(new_pos)
    bob.curr_pos = new_pos

print(f"Steps taken = {iter_count}\nPath = {path}")

Steps taken = 2
Path = [(0, 0), (9, 0), (9, 9)]


As expected, the toroidal grid makes it so that the best way to get from `(0, 0)` to `(9, 9)` is to wrap around the grid twice. I.e. move up once `(0 - 1) % 10 = 9` and move left once `(0 - 1) % 10 = 9`.

Note that the agent cannot simply find a path from some arbitrary start point to an arbitrary end point UNLESS it has first been trained on those points.